# Walidacja drugiego kamienia milowego
### Walidujący: Alicja Charuza, Mateusz Gałęziewski
### Walidowana grupa: Maciej Borkowski, Michał Chęć

Formą walidacji drugiego kamienia milowego będzie odpalenie dotychczas napisanego kodu na zbiorze testowym, a także sprawdzenie, jak przygotowane modele sobie na nim radzą.

In [1]:
# ładujemy potrzebne pakiety
import pandas as pd
import numpy as np
from tabulate import tabulate

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, SelectFromModel, SequentialFeatureSelector

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
import os
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

np.random.seed(42)

In [19]:
# wczytujemy całą ramkę danych
dataset = pd.read_csv("dataset_1.csv")
df = dataset.copy()

In [20]:
# dzielimy dane, tworzymy zbiór do ewaluacji i do testów
train_set, eval_set = train_test_split(df, test_size=0.3, random_state=42)
train_df, test_df = train_test_split(train_set, test_size=0.3, random_state=42)

In [21]:
# klasa do preprocessingu danych
class ColumnRemover(BaseEstimator, TransformerMixin):

    def __init__(self, threshold_constant, threshold_corr, n_info_vals):
        self.threshold_constant = threshold_constant
        self.threshold_corr = threshold_corr
        self.n_info_vals = n_info_vals
        self.columns_to_remove = []
        self.columns_to_keep = []

    def fit(self, X, y=None):
        # usuwanie zduplikowanych kolumn
        self.columns_to_remove.extend(X.loc[:, X.T.duplicated()].columns.tolist())

        # usuwanie stałych i prawie stałych kolumn
        for column in X.columns:
            if X[column].value_counts(normalize=True).iloc[0] >= self.threshold_constant:
                self.columns_to_remove.append(column)

        # usuwanie skorelowanych kolumn
        corr = X.corr()
        corr = corr[corr > self.threshold_corr]
        dependent_columns = corr.apply(lambda row: row[row > 0].index, axis=1)
        for j in range (len(dependent_columns)):
            for k in dependent_columns[j]:
                if k is not dependent_columns.index[j]:
                    if k not in dependent_columns.index[0:j]:
                        self.columns_to_remove.append(k)

        # usuwanie kolumn nie niosących informacji
        amount_of_ones = y[y == 1].shape[0]
        X = X.join(y)
        for column in X.columns:
            tmp = X.groupby(column)['target'].agg(['sum','count']).sort_values('sum',ascending = False).reset_index()
            if any(tmp[column] == 0) and (tmp.loc[tmp[column] == 0, 'sum'] > amount_of_ones - self.n_info_vals).bool():
                self.columns_to_remove.append(column)
        X.drop('target', axis=1, inplace=True)

        self.columns_to_keep = [col for col in X.columns if col not in self.columns_to_remove]

        return self

    def transform(self, X):
        return X[self.columns_to_keep]

In [23]:
# zarówno dla zmiennych dyskretnych i ciągłych stosujemy nasz transformator ColumnRemover z różnymi parametrami - w kolejnych krokach będziemy szukać najlepszej ich kombinacji

# dokonujemy kodowania one hot encoding zmiennych dyskretnych - traktujemy je jako kategoryczne
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse=False, dtype='int64'))])

# dokojumeny standaryzacji zmiennych ciągłych - w kolejnych krokach sprawdzimy, czy jest lepsza od normalizacji min_max
float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10)),
    ('min_max', StandardScaler())])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

In [24]:
x_train = train_df.drop('target', axis=1)
y_train = train_df.target
x_eval = eval_set.drop('target', axis=1)
y_eval = eval_set.target

In [27]:
def show_scores(clf, X, y):
    y_pred = clf.predict(X)
    y_pred_prob = clf.predict_proba(X)
    print(tabulate(confusion_matrix(y, y_pred), headers=['Predicted 0', 'Predicted 1'], tablefmt='orgtbl'))
    print()
    print(f'accuracy:              {round(accuracy_score(y, y_pred), 4)}')
    print(f'precision:             {round(precision_score(y, y_pred), 4)}')
    print(f'recall:                {round(recall_score(y, y_pred), 4)}')
    print(f'f1:                    {round(f1_score(y, y_pred), 4)}')
    print(f'roc_auc_discrete:      {round(roc_auc_score(y, y_pred), 4)}')
    print(f'roc_auc_continuous:    {round(roc_auc_score(y, y_pred_prob[:, 1]), 4)}')

## Regresja logistyczna

In [28]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))])

clf.fit(x_train, y_train)
show_scores(clf, x_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         16315 |          7245 |
|           209 |           731 |

accuracy:              0.6958
precision:             0.0916
recall:                0.7777
f1:                    0.164
roc_auc_discrete:      0.7351
roc_auc_continuous:    0.8131


In [29]:
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|          9888 |          4511 |
|           166 |           435 |

accuracy:              0.6882
precision:             0.0879
recall:                0.7238
f1:                    0.1568
roc_auc_discrete:      0.7053
roc_auc_continuous:    0.7837


Model regresji logistycznej ma problem z uczeniem się. Pomóc może zastowanie dodatkowych parametrów i redukcja kolumn.

## Drzewo decyzyjne

In [32]:
# zarówno dla zmiennych dyskretnych i ciągłych stosujemy nasz transformator ColumnRemover z różnymi parametrami - w kolejnych krokach będziemy szukać najlepszej ich kombinacji

# dokonujemy kodowania one hot encoding zmiennych dyskretnych - traktujemy je jako kategoryczne
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse=False, dtype='int64'))])

# jesteśmy przy drzewach decyzyjnych, więc nie musimy skalować cech
float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

In [33]:
clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', DecisionTreeClassifier(random_state=42, class_weight='balanced'))])

clf.fit(x_train, y_train)
show_scores(clf, x_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23557 |             3 |
|             0 |           940 |

accuracy:              0.9999
precision:             0.9968
recall:                1.0
f1:                    0.9984
roc_auc_discrete:      0.9999
roc_auc_continuous:    1.0


In [34]:
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         13852 |           547 |
|           530 |            71 |

accuracy:              0.9282
precision:             0.1149
recall:                0.1181
f1:                    0.1165
roc_auc_discrete:      0.5401
roc_auc_continuous:    0.5401


Porównując wyniki na zbiorze treningowym i testowym, widzimy, że drzewo decyzyjne się przeucza. Rozwiązaniem może być wybór maksymalnej głębokości i redukcja kolumn.

## Random Forest

In [36]:
int_transformer = Pipeline([
    ('int', ColumnRemover(0.9995, 0.99, 1)),
    ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse=False, dtype='int64'))])

float_transformer = Pipeline([
    ('float', ColumnRemover(0.9999, 0.99, 10))])

col_transformer = ColumnTransformer([
    ('int_pipe', int_transformer, make_column_selector(dtype_include=np.int64)),
    ('float_pipe', float_transformer, make_column_selector(dtype_include=np.float64))
])

In [37]:
rf = RandomForestClassifier(n_estimators=1000)

clf = Pipeline([
    ('preprocessing', col_transformer),
    ('model', rf)])

clf.fit(x_train, y_train)
show_scores(clf, x_train, y_train)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         23560 |             0 |
|             3 |           937 |

accuracy:              0.9999
precision:             1.0
recall:                0.9968
f1:                    0.9984
roc_auc_discrete:      0.9984
roc_auc_continuous:    1.0


Patrząc na same wyniki na zbiorze treningowym, możemy podejrzewać, że model bardzo się przeuczył. Sprawdźmy jak sprawa wygląda przy zbiorze testowym.

In [38]:
show_scores(clf, x_eval, y_eval)

|   Predicted 0 |   Predicted 1 |
|---------------+---------------|
|         14370 |            29 |
|           593 |             8 |

accuracy:              0.9585
precision:             0.2162
recall:                0.0133
f1:                    0.0251
roc_auc_discrete:      0.5056
roc_auc_continuous:    0.8017


Nasze podejrzenia okazały się prawdziwe, gdyż model zdecydowanie gorzej poradził sobie na zbiorze testowym. Trzeba zastosować dodatkowe parametry, aby uniknąć przeuczania się modelu.